In [19]:
from sentinelhub import SHConfig

config = SHConfig()
config.instance_id = '0347fe1d-150a-4ad7-9244-9069a9b25839'
#config.instance_id = '84fe96a5-c0dc-4a72-85ad-6831fb2e5c8b'
config.sh_client_id = '470b0865-7123-4b24-a062-cec503e427d8'
config.sh_client_secret = 'x~!ad,O(CGi?q(Bp~:OM395]_;HeO1/*?))}!+1W'
config.download_timeout_seconds = 1200
config.save()
if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

In [20]:
#pip install sentinelhub

In [21]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [22]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    Geometry,
    SentinelHubStatistical,
    SentinelHubStatisticalDownloadClient,
    SHConfig,
    parse_time,
)

In [23]:
def stats_to_df(stats_data):
    """Transform Statistical API response into a pandas.DataFrame"""
    df_data = []

    for single_data in stats_data["data"]:
        df_entry = {}
        is_valid_entry = True

        df_entry["interval_from"] = parse_time(single_data["interval"]["from"]).date()
        df_entry["interval_to"] = parse_time(single_data["interval"]["to"]).date()

        for output_name, output_data in single_data["outputs"].items():
            for band_name, band_values in output_data["bands"].items():
                band_stats = band_values["stats"]
                if band_stats["sampleCount"] == band_stats["noDataCount"]:
                    is_valid_entry = False
                    break

                for stat_name, value in band_stats.items():
                    col_name = f"{output_name}_{band_name}_{stat_name}"
                    if stat_name == "percentiles":
                        for perc, perc_val in value.items():
                            perc_col_name = f"{col_name}_{perc}"
                            df_entry[perc_col_name] = perc_val
                    else:
                        df_entry[col_name] = value

        if is_valid_entry:
            df_data.append(df_entry)

    return pd.DataFrame(df_data)

In [24]:
import geopandas as gpd
polygon = gpd.read_file('agri_46.geojson')
#polygon.geometry.values[0]
#for plot in polygon.geometry.values:

In [25]:
with open("statapi_evalscript.js", "r") as fp:
    features_evalscript = fp.read()

In [26]:
yearly_time_interval = "2015-01-01", "2022-12-31"

aggregation = SentinelHubStatistical.aggregation(
    evalscript=features_evalscript, time_interval=yearly_time_interval, aggregation_interval="P1D", resolution=(10, 10) 
)

calculations = {"default": {"statistics": {"default": {"percentiles": {"k": [5, 50, 95]}}}}}

'''
request = SentinelHubStatistical(
        aggregation=aggregation,
        input_data=[SentinelHubStatistical.input_data(DataCollection.SENTINEL2_L2A)],
        geometry=Geometry(polygon.geometry.values[0], crs=CRS(polygon.crs)),
        calculations=calculations,
        config=config,
    ) 
'''

features_requests = []
for geo_shape in polygon.geometry.values:
    request = SentinelHubStatistical(
        aggregation=aggregation,
        input_data=[SentinelHubStatistical.input_data(DataCollection.SENTINEL2_L2A)],
        geometry=Geometry(geo_shape, crs=CRS(polygon.crs)),
        calculations=calculations,
        config=config,
    )

    features_requests.append(request)

/home/maxim/.local/lib/python3.10/site-packages/sentinelhub/constants.py:126: SHUserWarning: sentinelhub-py supports only WGS 84 coordinate reference system with coordinate order lng-lat. Given pyproj.CRS(4326) has coordinate order lat-lng. Be careful to use the correct order of coordinates.
  warnings.warn(message, category=SHUserWarning)


In [27]:
'''
client = SentinelHubStatisticalDownloadClient(config=config)

features_stats = client.download(request.download_list[0])

len(features_stats)
'''

download_requests = [request.download_list[0] for request in features_requests]

client = SentinelHubStatisticalDownloadClient(config=config)

features_stats = client.download(download_requests)

len(features_stats)


46

In [28]:
#print(features_stats[0])

In [29]:
#ndvi_df = stats_to_df(features_stats)

ndvi_df = [stats_to_df(polygon_stats) for polygon_stats in features_stats]

for df, land_type in zip(ndvi_df, polygon["name"].values):
    df["name"] = land_type

ndvi_df = pd.concat(ndvi_df)

In [30]:
BANDS = DataCollection.SENTINEL2_L2A.bands
INDICES = ["NDVI", "NDVI_RE1", "NBSI", "CLP", "NDMI"]
STATISTICAL_QUANTITIES = ["mean", "min", "max", "stDev", "percentiles_5.0", "percentiles_50.0", "percentiles_95.0"]


for band in BANDS:
    for stat in STATISTICAL_QUANTITIES:
        column_name = f"bands_{band.name}_{stat}"
        column = ndvi_df[column_name]

        column = column / 10000.0

        ndvi_df[column_name] = column
        

for index in INDICES:
    for stat in STATISTICAL_QUANTITIES:
        column_name = f"indices_{index}_{stat}"
        column = ndvi_df[column_name]

        if stat == "stDev":
            column = column / 5000.0
        else:
            column = (column - 5000.0) / 5000.0

        ndvi_df[column_name] = column


In [31]:
#fig, ax = plt.subplots(figsize=(15, 8))

#ndvi_df.plot(ax=ax, x="interval_from", y="indices_NDVI_mean")

In [32]:


filtered_df = ndvi_df[['interval_from', 'indices_NDVI_mean', 'indices_NDMI_mean', 'indices_NBSI_mean', 'name']]

filtered_df.to_csv('filtered_df.csv')

In [37]:
polygon = gpd.read_file('agri_46.geojson')
print(polygon)

             name                                           geometry
0      winschoten  POLYGON ((7.09423 53.12906, 7.09121 53.12538, ...
1           sneek  POLYGON ((5.70158 53.04088, 5.69605 53.03779, ...
2           assen  POLYGON ((6.46739 52.96280, 6.46321 52.95847, ...
3          arnhem  POLYGON ((5.80134 51.94159, 5.80470 51.93637, ...
4         tilburg  POLYGON ((4.96834 51.54021, 4.96888 51.53289, ...
5      maastricht  POLYGON ((5.74985 50.84715, 5.74950 50.84506, ...
6      middelburg  POLYGON ((3.52742 51.50328, 3.52037 51.51019, ...
7      zoetermeer  POLYGON ((4.58442 52.00565, 4.56554 52.00048, ...
8         utrecht  POLYGON ((5.19966 52.03904, 5.19412 52.04373, ...
9       eindhoven  POLYGON ((5.28883 51.40995, 5.29019 51.40736, ...
10     roosendaal  POLYGON ((4.42785 51.55336, 4.42515 51.55441, ...
11      emmeloord  POLYGON ((5.68218 52.73155, 5.68211 52.72456, ...
12   wieringswerf  POLYGON ((5.00167 52.86342, 5.00064 52.85699, ...
13      hoogeveen  POLYGON ((6.514